In [1]:
from model.model import ConvModel, PreTrainer, show_results, HeadTrainer

import argparse
import torch
import torch.backends.cudnn as cudnn
from torchvision import models
from torchvision import transforms, datasets


D:\Programy\DataSpell 2022.2.3\DataspellProjects\SimCLRPytorch-main\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
    gpu_index = 0
else:
    device = torch.device('cpu')
    gpu_index = -1


In [3]:
color_jitter = transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)


composed_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([color_jitter], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.GaussianBlur(kernel_size=3),
      transforms.ToTensor()])

In [6]:
train_dataset_1 = datasets.MNIST('./datasets', train=True,
                               transform=composed_transforms,
                               download=True)
train_dataset_2 = datasets.MNIST('./datasets', download=True, train=True, transform=transforms.ToTensor())#ten nie ma augumentacji
g = torch.Generator()
g.manual_seed(0)

In [7]:
train_loader_1 = torch.utils.data.DataLoader(
    train_dataset_1, batch_size=256, shuffle=False,
     pin_memory=True, drop_last=True, generator=g)
train_loader_2 = torch.utils.data.DataLoader(train_dataset_2, batch_size=256, shuffle=True, pin_memory=True, drop_last=True, generator=g)

In [10]:
model = ConvModel(out_dim=10, dataset='MNIST')

#model.load_state_dict(torch.load('saved_models/model_final_cifar10_200_epochs.pth'))
model.load_state_dict(torch.load('saved_models/MNISTpoPRetreningu.pth'))


<All keys matched successfully>

In [8]:
torch.save(model.state_dict(), 'saved_models/MNISTpoPRetreningu.pth')

In [ ]:
#torch.load(model.state_dict(), 'saved_models/MNISTpoPRetreningu.pth')

In [11]:
model

ConvModel(
  (resnet_model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

In [12]:
model.backbone.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=512, out_features=128, bias=True),
    torch.nn.ReLU(inplace=True),
    # torch.nn.BatchNorm1d(128),
    torch.nn.Linear(in_features=128, out_features=10, bias=True),
)


In [13]:
model.backbone.requires_grad_(False)
model.backbone.fc.requires_grad_(True)

for parameter in model.backbone.named_parameters():
    print(f'{parameter[0]}: {parameter[1].requires_grad}')


conv1.weight: False
bn1.weight: False
bn1.bias: False
layer1.0.conv1.weight: False
layer1.0.bn1.weight: False
layer1.0.bn1.bias: False
layer1.0.conv2.weight: False
layer1.0.bn2.weight: False
layer1.0.bn2.bias: False
layer1.1.conv1.weight: False
layer1.1.bn1.weight: False
layer1.1.bn1.bias: False
layer1.1.conv2.weight: False
layer1.1.bn2.weight: False
layer1.1.bn2.bias: False
layer2.0.conv1.weight: False
layer2.0.bn1.weight: False
layer2.0.bn1.bias: False
layer2.0.conv2.weight: False
layer2.0.bn2.weight: False
layer2.0.bn2.bias: False
layer2.0.downsample.0.weight: False
layer2.0.downsample.1.weight: False
layer2.0.downsample.1.bias: False
layer2.1.conv1.weight: False
layer2.1.bn1.weight: False
layer2.1.bn1.bias: False
layer2.1.conv2.weight: False
layer2.1.bn2.weight: False
layer2.1.bn2.bias: False
layer3.0.conv1.weight: False
layer3.0.bn1.weight: False
layer3.0.bn1.bias: False
layer3.0.conv2.weight: False
layer3.0.bn2.weight: False
layer3.0.bn2.bias: False
layer3.0.downsample.0.weight: 

In [7]:
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader_1), eta_min=0,
                                                       last_epoch=-1)

with torch.cuda.device(gpu_index):
    simclr = PreTrainer(model=model, optimizer=optimizer, scheduler=scheduler, device=device, epochs=200, log_every_n_steps=100)
    simclr.train(train_loader_1, shuffle=True)


  0%|          | 0/234 [00:00<?, ?it/s]D:\Programy\DataSpell 2022.2.3\DataspellProjects\SimCLRPytorch-main\venv\lib\site-packages\torch\optim\lr_scheduler.py:807: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
100%|██████████| 234/234 [01:06<00:00,  3.53it/s]


In [14]:
head_dataset = datasets.MNIST('./datasets', download=True, train=False, transform=transforms.ToTensor()) #update of names to more suitable
head_loader = torch.utils.data.DataLoader(head_dataset, batch_size=256, shuffle=True, pin_memory=True, drop_last=True, generator=g)

In [16]:
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader_1), eta_min=0,
                                                       last_epoch=-1)

In [ ]:
train_size = int(0.8 * len(head_dataset))
test_size = len(head_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(head_dataset, [train_size, test_size])
trainer = HeadTrainer(train_dataset, test_dataset, batch_size=256)
history = trainer.train(model, optimizer, n_epochs=200)
show_results(model=history)

In [38]:
import logging
from torch.utils.tensorboard import SummaryWriter
import os
from model.model import accuracy
from tqdm import tqdm

total = 0
running_accuracy = 0
n_iter = 0
writer = SummaryWriter()
logging.basicConfig(filename=os.path.join(writer.log_dir, 'training.log'), level=logging.DEBUG)
criterion = torch.nn.CrossEntropyLoss().to(device)

with torch.no_grad():
    for images, labels in tqdm(head_loader):
        images = torch.cat((images, images), dim=0)
        labels = torch.cat((labels, labels), dim=0)

        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        top1, top5 = accuracy(output, labels, topk=(1, 5))
        writer.add_scalar('loss', loss, global_step=n_iter)
        writer.add_scalar('acc/top1', top1[0], global_step=n_iter)
        writer.add_scalar('acc/top5', top5[0], global_step=n_iter)
        n_iter += 1
        print(top1)

 10%|█         | 4/39 [00:00<00:03, 11.32it/s]

tensor([32.0312], device='cuda:0')
tensor([40.2344], device='cuda:0')
tensor([37.5000], device='cuda:0')
tensor([37.1094], device='cuda:0')
tensor([38.6719], device='cuda:0')
tensor([35.1562], device='cuda:0')


 26%|██▌       | 10/39 [00:00<00:01, 17.73it/s]

tensor([32.0312], device='cuda:0')
tensor([35.5469], device='cuda:0')
tensor([37.5000], device='cuda:0')
tensor([36.7188], device='cuda:0')
tensor([37.1094], device='cuda:0')


 33%|███▎      | 13/39 [00:00<00:01, 19.17it/s]

...

tensor([35.1562], device='cuda:0')
tensor([32.4219], device='cuda:0')
tensor([38.2812], device='cuda:0')
tensor([36.7188], device='cuda:0')
tensor([35.5469], device='cuda:0')


 87%|████████▋ | 34/39 [00:01<00:00, 22.57it/s]

tensor([33.2031], device='cuda:0')
tensor([35.5469], device='cuda:0')
tensor([33.2031], device='cuda:0')
tensor([41.0156], device='cuda:0')
tensor([33.9844], device='cuda:0')
tensor([29.6875], device='cuda:0')


100%|██████████| 39/39 [00:01<00:00, 20.01it/s]

tensor([39.0625], device='cuda:0')
tensor([33.9844], device='cuda:0')


In [ ]:
torch.save(model.state_dict(), 'saved_models/model_ultimate_MNIST1s.pth')

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader_2), eta_min=0,
                                                       last_epoch=-1)
with torch.cuda.device(gpu_index):\
    simclr = PreTrainer(model=model, optimizer=optimizer, scheduler=scheduler, device=device, epochs=200, log_every_n_steps=100)
    simclr.train(train_loader_2, shuffle=False)
